# Analysis of LLM Labels

- Load LLM-labelled data
- Calculate the ratio of posts that are labelled prediction and non-predicton

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# print(notebook_dir)
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [6]:
pd.set_option('max_colwidth', 800)

In [3]:
path_to_data = "../data/open_measures/bluesky_all_llm_labels/bluesky-v10.json"
full_path = os.path.join(notebook_dir, path_to_data)
df = pd.read_json(full_path)
df

,sentence,model,label,meta_data,majority_vote
0,Wow! Nepal confirmed as one of 16 finalists fo...,gemma2-9b-it,not a prediction,"{'data_range_per_file': 0, '$type': 'app.bsky....",not a prediction
1,Wow! Nepal confirmed as one of 16 finalists fo...,llama-3.1-8b-instant,prediction,"{'data_range_per_file': 0, '$type': 'app.bsky....",not a prediction
2,Wow! Nepal confirmed as one of 16 finalists fo...,llama-3.3-70b-versatile,not a prediction,"{'data_range_per_file': 0, '$type': 'app.bsky....",not a prediction
3,Wow! Nepal confirmed as one of 16 finalists fo...,meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 0, '$type': 'app.bsky....",not a prediction
4,Sad! AFC bans Nepal from U16 Asian Championshi...,gemma2-9b-it,not a prediction,"{'data_range_per_file': 1, '$type': 'app.bsky....",not a prediction
...,...,...,...,...,...
798,"3️⃣: Super Bowl LVII\n""Super Bowl LVII was an ...",meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 198, '$type': 'app.bsk...",not a prediction
799,Where do Reid and Mahomes sit among QB/coachin...,gemma2-9b-it,prediction,"{'data_range_per_file': 199, '$type': 'app.bsk...",prediction
800,Where do Reid and Mahomes sit among QB/coachin...,llama-3.1-8b-instant,prediction,"{'data_range_per_file': 199, '$type': 'app.bsk...",prediction
801,Where do Reid and Mahomes sit among QB/coachin...,llama-3.3-70b-versatile,not a prediction,"{'data_range_per_file': 199, '$type': 'app.bsk...",prediction


In [7]:
filt_prediction = (df['majority_vote'] == 'prediction')
df[filt_prediction]

,sentence,model,label,meta_data,majority_vote
8,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",gemma2-9b-it,not a prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction
9,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",llama-3.1-8b-instant,prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction
10,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",llama-3.3-70b-versatile,prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction
11,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoac